In [ ]:
#W = 27 cm
### SPEED PROFILE WHEN density varies

#load data

#particle density = 0.008 (particle alone)
datafolder0008= 'W:/Banquise/Matilde/vary_density/8V/0_008/data/'

vel_map0008, particle_map0008 = read(glob.glob(datafolder0008+'*vel_map*.pkl')[0])['vel_map'], read(glob.glob(datafolder0008+'*part*.pkl')[0])['part_map']


#particle density = 0.1, 12 particles
datafolder01 = 'W:/Banquise/Matilde/vary_density/8V/0_1/data/'
vel_map01, particle_map01 = read(glob.glob(datafolder01+'*vel_map*.pkl')[0])['vel_map'], read(glob.glob(datafolder01+'*part*.pkl')[0])['part_map']


#particle density = 0.2, 24 particles
datafolder02 = 'W:/Banquise/Matilde/vary_density/8V/0_2/data/'
vel_map02, particle_map02 = read(glob.glob(datafolder02+'*vel_map*.pkl')[0])['vel_map'], read(glob.glob(datafolder02+'*part*.pkl')[0])['part_map']


#particle density = 0.3, 35 particles
datafolder03 = 'W:/Banquise/Matilde/vary_density/8V/0_3/data/'
vel_map03, particle_map03 = read(glob.glob(datafolder03+'*vel_map*.pkl')[0])['vel_map'], read(glob.glob(datafolder03+'*part*.pkl')[0])['part_map']


#particle density = 0.4, 47 particles
datafolder04 = 'W:/Banquise/Matilde/vary_density/8V/0_4/data/'
vel_map04, particle_map04 = read(glob.glob(datafolder04+'*vel_map*.pkl')[0])['vel_map'], read(glob.glob(datafolder04+'*part*.pkl')[0])['part_map']

#particle density = 0.48 , 57 particles
datafolder048 = 'W:/Banquise/Matilde/vary_density/8V/0_48/data/'
vel_map048, particle_map048 = read(glob.glob(datafolder048+'*vel_map*.pkl')[0])['vel_map'], read(glob.glob(datafolder048+'*part*.pkl')[0])['part_map']


# parameters
NX, NY = 30,20
dx, dy = xdim/NX, ydim/NY

W = 27 #cm
r = 2 #cm
alim = 8#V
r0 = 15.8 #cm
k = 0.516 #conversion factor of motors : V to rad/s
o = k*alim #rad/s

U = k*r0*alim #cm/s
Ums = U*1e-2 #m/s
cm2px = xdim/55
Upx = U*cm2px #px/s

x = np.array([dy*i for i in np.linspace(0, NY-2, NY-2, endpoint = True)]) # /\ x axis corresponds to y variable IRL !
x = x/ydim # adim. space variable


density =  [0.008, 0.1, 0.2, 0.3,0.4, 0.48] 
velocities = [vel_map0008, vel_map01, vel_map02, vel_map03,vel_map04, vel_map048] 
particlenb = [1,12,24,35,47,57] 

part_map =  [particle_map0008, particle_map01,particle_map02, particle_map03,particle_map04, particle_map048] 
colors =  ['green', 'blue', 'red', 'purple', 'orange', 'black'] 

# density =  [0.008,0.1]
# velocities =  [vel_map0008,vel_map01] 
# particlenb = [1,12] 

# part_map =  [particle_map0008,particle_map01] 
# colors =  ['green', 'blue'] 

### fitting stuff

def func(e, aRe, C):
    return C*(1- (np.log(1+ aRe*(1-e)))/(np.log(1+aRe))) 

p0 = [8000000, 0.1]
p1= [9000000*100, 0.004]






##speed profile

fig,ax  = plt.subplots(figsize = (10,8))

for vel_map, d,n, pmap, c in zip(velocities, density, particlenb, part_map, colors) :

    #get dimensions
    frmd, yd, xd, _ = vel_map.shape
    
    # convert NaN to 0 for speed values if needed
    velocity_map0 = np.copy(vel_map)
    where_are_NaNs = np.isnan(velocity_map0)
    velocity_map0[where_are_NaNs] = 0

    vx0 = velocity_map0[:,:,:,0]
    vx0_weighted = (vx0*pmap)/n # weight every box value by local particle density
    
    #time average
    ux0 = np.nanmean(vx0_weighted, axis = 0)
    #ux0 = ux0[:, 0:xd//4 ] # aerage only over specific part of the disk

    #average over x position
    avg_vx = np.mean(ux0, axis = 1)
    avg_vx = avg_vx/Upx
    y = avg_vx[1:-1]
   
    #plot data 
    ax.plot(x,y, marker = '+', ms = 12, mew= 3,label  = r'$\Phi =$ {}'.format(d), color = c)
        
 
        
    
# background
for i in x:
    plt.axvline(i, linestyle = '--', color = 'gray', lw = 0.4)
        
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True) 
formatter.set_powerlimits((-1,1)) 
ax.yaxis.set_major_formatter(formatter)
ax.set_title('Profil de vitesse moyen (W = {}cm, '.format(W)+'rayon = {}cm, '.format(r) + r'$\Omega$ = {:.2f} rad/s)'.format(o), fontsize = 25)
ax.set_xlabel(r'$\xi$',fontsize = 22)
ax.set_ylabel(r'$\frac{V_x}{V_0}$ ', fontsize = 22)
ax.tick_params(labelsize=20)
ax.yaxis.offsetText.set_fontsize(20)
ax.legend(loc = 'best', fontsize = 17)



### speed profile + fit

fig, ax = plt.subplots(figsize = (10,8))


fit_params = [] # initialize fitted params list
x4fitplot = []

for vel_map, d,n, pmap, c in zip(velocities, density, particlenb, part_map, colors) :
    frmd, yd, xd, _ = vel_map.shape
    
# convert NaN to 0 for speed values if needed
    velocity_map0 = np.copy(vel_map)
    where_are_NaNs = np.isnan(velocity_map0)
    velocity_map0[where_are_NaNs] = 0

    vx0 = velocity_map0[:,:,:,0]
    vx0_weighted = (vx0*pmap)/n # weight every box value by local particle density
    
    #time average
    ux0 = np.nanmean(vx0_weighted, axis = 0)
    #ux0 = ux0[:, 3*xd//4 :  ] # delect only specific part of the disk
    
    #average over x position
    avg_vx = np.mean(ux0, axis = 1)
    avg_vx = avg_vx/Upx
    y = avg_vx[1:-1]
    
    #find right part of the curb -> fit is symetric, focus on one part
    cond = np.where(x>0.4)
    xdata = x[cond] 
    ydata = y[cond]
    xdata = xdata[:-2] #delete extreme border points
    ydata = ydata[:-2]
    
    # reset data : x and y so that start at 0
    ydata-= ydata[0] 
    xdata-= xdata[0]

 
    #interpolate
    axis_list = np.transpose([xdata.reshape(-1,1), ydata.reshape(-1,1)])
    
    midx, midy = interp(*axis_list, plot = False)
    midx, midy = np.array(midx), np.array(midy)
    print(midx[0])

     
    #plot data on selected area
    ax.yaxis.set_major_formatter(formatter)
    ax.plot(xdata,ydata, marker = '+', ms = 12, mew= 3,label  = r'$\Phi =$ {}'.format(d), color = c)
        
    #fit interpolated data
#     sigma = np.ones(len(midx))
#     sigma[[0, -1]] = 0.01
    popt, pcov = curve_fit(func, midx, midy, p0 = p0,  bounds = ([8e5, 0], [8e8, 10]),   maxfev = 2000)
    ax.yaxis.set_major_formatter(formatter)
    ax.plot(midx, func(midx, *popt), ls='--', color = c)
    fit_params.append(popt)
    x4fitplot.append(midx)
      
        
    
# background
for i in xdata:
    plt.axvline(i, linestyle = '--', color = 'gray', lw = 0.4)
        
ax.set_title('Profil de vitesse moyen et fit (W = {}cm, '.format(W)+'rayon = {}cm, '.format(r) + r'$\Omega$ = {:.2f} rad/s)'.format(o), fontsize = 25)
ax.set_xlabel(r'$\xi$',fontsize = 22)
ax.set_ylabel(r'$\frac{V_x}{V_0}$', fontsize = 22)
ax.tick_params(labelsize=20)
ax.yaxis.offsetText.set_fontsize(20)
ax.legend(loc = 'best', fontsize = 17)



### fit only
fig, ax = plt.subplots(figsize = (10,8))

for xplot, params, c, d in zip(x4fitplot, fit_params, colors, density):
    #full fit plotted
    ax1 = plt.subplot(1,2,1)
    ax1.plot(xplot, func(xplot, *params), 
             label = r'$ \Phi = {}$, '.format(d) + 'fit: aRe=%5.3e, C=%5.3e' % tuple(params), color = c )
    ax1.legend(loc = 'best', fontsize = 10)
    ax1.tick_params(labelsize=20)
    ax1.set_ylabel('Fit', fontsize = 22)
    ax1.set_xlabel('Densité', fontsize = 22)
    ax1.yaxis.set_major_formatter(formatter)
    ax1.yaxis.offsetText.set_fontsize(20)
        
    plt.tight_layout()
    
    # plot second fitted coeff
    ax2 = plt.subplot(1,2,2)
    aRe = params[0]
    nu = Ums*W*(10**-2)/aRe
    ax2.scatter(d, aRe, color = c)
    ax2.tick_params(labelsize=20)
    ax2.set_xlabel('Densité', fontsize = 22)
    ax2.set_ylabel(r'$\alpha\mathrm{Re}$', fontsize = 22)
    ax2.yaxis.offsetText.set_fontsize(20)
    
    
    